In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from skrebate import ReliefF
from sklearn.preprocessing import StandardScaler



In [2]:
# Charger les données
data = pd.read_csv('C:/Users/antho/Documents/MEMOIRE_M2/data_final.csv')

# Identifier les colonnes contenant des valeurs NA
na_columns = data.columns[data.isna().any()].tolist()

# Supprimer les colonnes avec des NA
data_cleaned = data.dropna(axis=1)

# Séparer les caractéristiques et les étiquettes
X = data_cleaned.drop('label', axis=1)  # données issues desc caractéristiques
y = data_cleaned['label'].apply(lambda X: 1 if X in ['fog', 'transitionFog'] else 0) # labels transofmré en cible et non cible



# Standardisation des caractéristiques

In [3]:

# Création d'un objet StandardScaler
scaler = StandardScaler()

# Adaptation du scaler aux données (calcul de la moyenne et de l'écart-type)
scaler.fit(X)  # X est votre DataFrame avec les caractéristiques

# Transformation des données de caractéristsiques
X_scaled = scaler.transform(X)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# VC pour créer des données FOG et enlever des données NOFOG

In [8]:
from imblearn.pipeline import Pipeline as ImbPipeline  # Importez le Pipeline d'imbalanced-learn
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Définissez les paramètres de sur-échantillonnage et de sous-échantillonnage
smote_params = {'smote__sampling_strategy': [0.25, 0.4, 0.5, 0.6, 0.75, 1.0]}
under_params = {'under__sampling_strategy': [0.25, 0.4, 0.5, 0.6, 0.75, 1.0]}

# Configuration du pipeline avec imblearn's Pipeline
model = RandomForestClassifier(random_state=42)
over = SMOTE(random_state=42)
under = RandomUnderSampler(random_state=42)
pipeline = ImbPipeline([
    ('over', over),
    ('under', under),
    ('model', model)
])


In [9]:
from sklearn.model_selection import GridSearchCV

# Combinaison des grilles de paramètres
params = {**smote_params, **under_params}

# Configuration de GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=params, scoring='accuracy', cv=5)  # cv=5 pour une validation croisée à 5 plis

# Entraînement et recherche des meilleurs paramètres
grid_search.fit(X_train, y_train)

# Affichage des meilleurs paramètres et de la meilleure score
print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleur score :", grid_search.best_score_)



ValueError: Invalid parameter smote for estimator Pipeline(steps=[('over', SMOTE(random_state=42)),
                ('under', RandomUnderSampler(random_state=42)),
                ('model', RandomForestClassifier(random_state=42))]). Check the list of available parameters with `estimator.get_params().keys()`.

In [4]:
# Division des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:

# Initialisation de ReliefF
relief = ReliefF(n_neighbors=20)  # Vous pouvez ajuster 'n_neighbors' selon la taille de votre jeu de données et les spécificités du problème

# Entraînement de ReliefF sur les données d'entraînement
relief.fit(X_train.to_numpy(), y_train.to_numpy())

# Obtention des scores de l'importance des caractéristiques
feature_scores = relief.feature_importances_

# Création d'un DataFrame pour mieux visualiser les scores des caractéristiques
feature_importance_df = pd.DataFrame(feature_scores, index=X_train.columns, columns=["Importance"])
feature_importance_df.sort_values("Importance", ascending=False, inplace=True)